In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# ! pip install shap

In [ ]:
%cd ..

In [ ]:
data_path = "data/Quote_and_Order_Data_2025_01_10.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import hvplot.pandas
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import shap

hvplot.extension('plotly')
output_notebook()
%matplotlib inline
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')
shap.initjs()

In [ ]:
data = pd.read_csv(
    data_path, 
    delimiter=",", 
    encoding="latin1", 
    # on_bad_lines="warn", 
    on_bad_lines='warn', 
    engine='python'
    )

# Data cleaning

In [ ]:
data = data[data["Seller Price"] > 0]

# Exploration

In [ ]:
data.head()

In [ ]:
data.columns

## EDA Stats

In [ ]:
# data["Quote/Order Number"].nunique()
# data.groupby("Quote/Order Number")["Quote/Order Status"].agg("first").value_counts(normalize=True)
# data.groupby("Quote/Order Number")["Quote/Order Status"].agg("first")
# pd.to_datetime(data["Quote/Order Created Date"], dayfirst=True).min(), pd.to_datetime(data["Quote/Order Created Date"],dayfirst=True).max()

In [ ]:
data.isna().mean()

In [ ]:
len(data)

In [ ]:
data.columns

## Column of interest

In [ ]:
SELECTED_COLUMNS = [
    "Quote/Order Number",
    "QTY",
    "Seller Price",
    "Buyer Price",
    "Delivery Address Region",
    "Buyer Company Region",
    "Quote/Order Status",
    "Quote/Order Created Date",
    "Delivery_method",
    "Supplier Name",
    "Buyer Name",
    "Common Name",
    "product_id",
    ]

In [ ]:
data = data[SELECTED_COLUMNS]

In [ ]:
data.head(5).to_dict("index")

### Feature engineering

In [ ]:
def feature_enngineeing(data):
    data["Seller Total"] = data["Seller Price"] * data["QTY"]
    data["Quote/Order Created Date"] = pd.to_datetime(data["Quote/Order Created Date"], dayfirst=True).dt.date
#     data["Quote/Order Status"] = data["Quote/Order Status"].replace(
#     {
#         "Quote Returned":0,
#         "Order Waiting for Pickup or Delivery":1,
#         "Order Complete":1,
#     }
# )
    return data

In [ ]:
data["Quote/Order Status"] = data["Quote/Order Status"].replace(
    {
        "Order Waiting for Pickup or Delivery":"Quote Returned",
    }
)

In [ ]:
data["Seller Total"] = data["Seller Price"] * data["QTY"]

In [ ]:
data["Quote/Order Created Date"] = pd.to_datetime(data["Quote/Order Created Date"], dayfirst=True).dt.date

## Compute markup

In [ ]:
data["line_item_markup_pct"] = (data["Buyer Price"] / (data["Seller Price"]) - 1.0)*100

In [ ]:
data["line_item_markup_pct"].describe()

In [ ]:
data["line_item_markup_pct"].quantile(0.95)

In [ ]:
# data[data["markup"] > 100][["Buyer Price", "Seller Price", 'Latin Name', "Common Name", "Quote/Order Created Date", "Buyer Name", "Supplier Name"]]

## Profile report

In [ ]:
# profile = ProfileReport(data, title="Profiling Report")
# profile

## Some visualization

### Distribution of markup across buyer region

In [ ]:
from turtle import st
import seaborn as sns
import numpy as np  

bins = np.linspace(0,100, 10).tolist() 
# + [2.0, 10.0, 100.0]
sns.displot(data, x="line_item_markup_pct", bins=bins, stat="proportion")

In [ ]:
# fig, ax = plt.subplots()
# sns.boxplot(data=data, x="Quote/Order Status", y="line_item_markup")

In [ ]:
data.head()

In [ ]:
original_data = data.copy()

In [ ]:
def compute_quote_global_markup(rows):
    score = ((rows["Buyer Price"]*rows["QTY"]).sum() / (rows["Seller Price"]*rows["QTY"]).sum() - 1.0) * 100
    return score

ORDER_ID_COLUMN = "Quote/Order Number"
MARKUP_COLUMN = "quote_markup_pct"
ORDER_DATE = "Quote/Order Created Date"
TOTAL_QUOTE_ITEMS_COLUMN = "QTY"
SERVICE_COST = "Seller Total"
QUOTE_STATUS = "Quote/Order Status"
REGION_COLUMN = "Buyer Company Region"
CUSTOMER_COLUMN = "Buyer Name"
TOTAL_ITEMS_COLUMN = "Total Items"
AGG_MAPPING = {
        # MARKUP_COLUMN: "first",
        ORDER_DATE: "first",
        TOTAL_QUOTE_ITEMS_COLUMN: "sum",
        SERVICE_COST: "sum",
        QUOTE_STATUS: "first",
        REGION_COLUMN: "first",
        CUSTOMER_COLUMN: "first",
    }

global_markups = data.groupby(ORDER_ID_COLUMN).apply(compute_quote_global_markup,include_groups=False)

per_quote_markup = data.groupby(ORDER_ID_COLUMN).agg(AGG_MAPPING)
per_quote_markup[MARKUP_COLUMN] = global_markups

In [ ]:
def quote_details_to_global_quote(data):
    global_quote = data.groupby(ORDER_ID_COLUMN).agg(AGG_MAPPING)
    global_quote = global_quote.rename({TOTAL_QUOTE_ITEMS_COLUMN:TOTAL_ITEMS_COLUMN},axis=1)
    global_quote[ORDER_DATE] = pd.to_datetime(per_quote_markup[ORDER_DATE])
    return global_quote

In [ ]:
per_quote_markup = per_quote_markup.rename({TOTAL_QUOTE_ITEMS_COLUMN:TOTAL_ITEMS_COLUMN},axis=1)
per_quote_markup[ORDER_DATE] = pd.to_datetime(per_quote_markup[ORDER_DATE])
per_quote_markup["year"] = per_quote_markup[ORDER_DATE].dt.year.astype(str)
per_quote_markup["month"] = per_quote_markup[ORDER_DATE].dt.month.astype(str)

In [ ]:
per_quote_markup.columns

In [ ]:
# df.hvplot(
#     kind='bar',
#     x='Buyer Company Region',
#     y=['markup'],
#     legend='bottom_right',
#     widget_location='bottom',
# # )

In [ ]:
bins = np.linspace(0,100, 10).tolist() 
# + [2.0, 10.0, 100.0]
sns.displot(per_quote_markup, x=MARKUP_COLUMN, bins=bins, stat="proportion")

In [ ]:
per_quote_markup[MARKUP_COLUMN].describe()

In [ ]:
per_quote_markup[MARKUP_COLUMN].quantile(0.95)

In [ ]:
per_quote_markup[per_quote_markup[QUOTE_STATUS] == "Order Complete"][MARKUP_COLUMN].quantile(0.95)

In [ ]:
# per_quote_markup.hvplot.explorer()

In [ ]:
top_10_regions = per_quote_markup[per_quote_markup["Buyer Company Region"].isin(per_quote_markup["Buyer Company Region"].value_counts().head(10).index)]
sns.barplot(top_10_regions, x=REGION_COLUMN, y=MARKUP_COLUMN, estimator="mean", errorbar=None, hue=QUOTE_STATUS, palette=["cyan", "blue"])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Buyer Company Region', fontsize=14)  # Increase x-axis label font size
plt.ylabel('quote_markup_pct', fontsize=14)  #

In [ ]:
per_quote_markup[[MARKUP_COLUMN, SERVICE_COST, TOTAL_ITEMS_COLUMN]].corr()

In [ ]:
TOP_10_REGIONS = [
    "Florida",
    "Texas",
    "New York",
    "Georgia",
    "South Carolina",
    "California",
    "Tennessee",
    "Louisiana",
    "North Carolina",
    "Mississippi",
    "Maryland",
    "New Jersey",
    "Kentucky",
]
per_quote_markup = per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS)]

# Quote status classification using quote's description

In [ ]:
data = per_quote_markup.copy(True)
data = data[
    [
        QUOTE_STATUS,
        # "year",
        # "month",
        SERVICE_COST,
        TOTAL_ITEMS_COLUMN,
        MARKUP_COLUMN,
        REGION_COLUMN,
    ]
]
data[QUOTE_STATUS] = data[QUOTE_STATUS].replace(
    {
        "Quote Returned":0,
        "Order Waiting for Pickup or Delivery":1,
        "Order Complete":1,
    }
)
data = data[data["Buyer Company Region"].isin(data["Buyer Company Region"].value_counts().head(10).index)]

In [ ]:
data[QUOTE_STATUS].value_counts()

In [ ]:
accepted_quotes = data[data[QUOTE_STATUS] == 1]
refused_quotes = data[data[QUOTE_STATUS] == 0].sample(len(accepted_quotes)*2)
data = pd.concat([accepted_quotes, refused_quotes], ignore_index=True).sample(frac=1.0)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Custom transformation function
def log_transform(x):
    return np.log1p(x)  # log1p is log(1 + x), useful to handle zero values

# FunctionTransformer
log_transformer = FunctionTransformer(log_transform, validate=True)

# Features and target
X = data.drop(QUOTE_STATUS, axis=1)
y = data[QUOTE_STATUS]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [TOTAL_ITEMS_COLUMN, SERVICE_COST, MARKUP_COLUMN]),
        ('cat', OneHotEncoder(handle_unknown="ignore", sparse_output=False), [REGION_COLUMN])
        # ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), [REGION_COLUMN])
    ],
    # remainder="passthrough",
    remainder="drop",
).set_output(transform="pandas")

# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier()),
    # ('classifier', HistGradientBoostingClassifier(
    #         random_state=42, 
    #         categorical_features=[f"cat__{REGION_COLUMN}"]
    #     )
    #                               ),
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
pipeline.classes_

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 
import matplotlib.pyplot as plt


# Predictions 
y_pred = pipeline.predict(X_test) 
# Confusion Matrix s
cm = confusion_matrix(y_test, y_pred) 
# Display Confusion Matrix 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["refused_quotes", "accepted quotes"]) 
disp.plot(cmap=plt.cm.Blues) 
plt.show()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    pipeline, X_test, y_test, name="Classifier", plot_chance_level=False
)
_ = display.ax_.set_title("Precision-Recall curve")

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score
# Predict probabilities on the test set
y_prob = pipeline.predict_proba(X_test)[:, 1]

# Get precision, recall, and thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

# Initialize variables to store the best threshold and best F1 score
best_threshold = 0.0
best_f1_score = 0.0

# Iterate over thresholds to find the best one based on F1 score
for threshold in thresholds:
    y_pred = (y_prob >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    
    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = threshold

In [ ]:
best_threshold

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 
import matplotlib.pyplot as plt


# Predictions 
y_pred = pipeline.predict(X_train) 
# Confusion Matrix s
cm = confusion_matrix(y_train, y_pred) 
# Display Confusion Matrix 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_) 
disp.plot(cmap=plt.cm.Blues) 
plt.show()

In [ ]:
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score

# Predict probabilities on the test set
y_prob = pipeline.predict_proba(X_test)[:, 1]

# Compute Brier score
brier_score = brier_score_loss(y_test, y_prob)
entropy_score = log_loss(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

print(f"Brier score: {brier_score}")
print(f"Entropy score: {entropy_score}")
print(f"AUC score: {auc_score}")

## Feature importance

In [ ]:
len(X_test)

In [ ]:
len(X_train)

In [ ]:
processed_train = preprocessor.transform(X_train)

model = pipeline.steps[1][1]
df = processed_train.sample(3000)
processed_X_test = preprocessor.transform(X_test)
explainer = shap.explainers.Permutation(model.predict_proba, df)
shap_values = explainer(processed_X_test)
shap_values = shap_values[..., 1]

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
X_test.head()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
features = [2]
PartialDependenceDisplay.from_estimator(pipeline, X_test, features,
    kind='average')
plt.title("Partial dependence plot",  fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=18)
plt.xlabel("Markup value", fontsize=18)
plt.ylabel("Partial dependence", fontsize=18)